# Combine basin-scale maps for global dataset
- Include AOU and solubility to the O2 dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import gsw
import os
import pandas as pd
from multiprocessing import pool

In [2]:
def set_version(alg,argo,vin,TSdata):
    version = [f'{alg}.{argo}.1.{TSdata}.{vin}.4',
         f'{alg}.{argo}.2.{TSdata}.{vin}.4',
         f'{alg}.{argo}.3.{TSdata}.{vin}.4',
         f'{alg}.{argo}.4.{TSdata}.{vin}.4',
         f'{alg}.{argo}.5.{TSdata}.{vin}.4']
    gver=f'{alg}.{argo}.G.{TSdata}.{vin}.4'
    return version,gver

In [3]:
os.system('echo $USER > userid')
usrid=np.genfromtxt('userid',dtype='<U16')
os.system('rm userid')
#
diro2=f'/glade/derecho/scratch/{usrid}/ML4O2_results/'

In [4]:
TSdata = 4
#
if TSdata == 1:
    dirTS='/glade/campaign/univ/ugit0034/EN4/L09_20x180x360/'
    endyear=2020
    rollx=0
    zmax=1000
elif TSdata == 2:
    dirTS='/glade/campaign/univ/ugit0034/ORAS4/TSN2/'
    endyear=2017
    rollx=1
    zmax=1000
elif TSdata == 4:
    dirTS='/glade/campaign/univ/ugit0034/EN4/C14/'
    endyear=2025
    rollx=0
    zmax=5500

In [5]:
if TSdata == 2:
    ds=xr.open_dataset(dirTS+'ORAS4_TSN2_196501.nc')
elif TSdata == 1:
    ds=xr.open_dataset(dirTS+'EN4_TSN2_L09_180x360_196501.nc')
elif TSdata == 4:
    ds=xr.open_dataset(dirTS+'EN4_C14_180x360_196501.nc')
#
if rollx==1:
    ds.coords['lon'] = (ds.coords['lon'] + 180) % 360 - 180
    ds = ds.sortby(ds.lon)
#
x1 = ds.coords['lon']
y1 = ds.coords['lat']
z1 = ds.coords['depth'].sel(depth=slice(0,zmax))
t1 = ds.coords['time'].sel(time=slice('1965-01',f'{endyear}-12'))
Nx1= np.size(x1)
Ny1= np.size(y1)
Nz1= np.size(z1)
Nt1= np.size(t1)
#ds.CT[0,0,:,:].plot()
print(Nx1,Ny1,Nz1,Nt1)
print(z1)
#plt.show()

360 180 102 1
<xarray.DataArray 'depth' (depth: 102)>
array([0.00e+00, 5.00e+00, 1.00e+01, 1.50e+01, 2.00e+01, 2.50e+01, 3.00e+01,
       3.50e+01, 4.00e+01, 4.50e+01, 5.00e+01, 5.50e+01, 6.00e+01, 6.50e+01,
       7.00e+01, 7.50e+01, 8.00e+01, 8.50e+01, 9.00e+01, 9.50e+01, 1.00e+02,
       1.25e+02, 1.50e+02, 1.75e+02, 2.00e+02, 2.25e+02, 2.50e+02, 2.75e+02,
       3.00e+02, 3.25e+02, 3.50e+02, 3.75e+02, 4.00e+02, 4.25e+02, 4.50e+02,
       4.75e+02, 5.00e+02, 5.50e+02, 6.00e+02, 6.50e+02, 7.00e+02, 7.50e+02,
       8.00e+02, 8.50e+02, 9.00e+02, 9.50e+02, 1.00e+03, 1.05e+03, 1.10e+03,
       1.15e+03, 1.20e+03, 1.25e+03, 1.30e+03, 1.35e+03, 1.40e+03, 1.45e+03,
       1.50e+03, 1.55e+03, 1.60e+03, 1.65e+03, 1.70e+03, 1.75e+03, 1.80e+03,
       1.85e+03, 1.90e+03, 1.95e+03, 2.00e+03, 2.10e+03, 2.20e+03, 2.30e+03,
       2.40e+03, 2.50e+03, 2.60e+03, 2.70e+03, 2.80e+03, 2.90e+03, 3.00e+03,
       3.10e+03, 3.20e+03, 3.30e+03, 3.40e+03, 3.50e+03, 3.60e+03, 3.70e+03,
       3.80e+03, 3.90e

In [7]:
yrs=np.arange(1965,endyear+1,1)
yrs

array([1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975,
       1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986,
       1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997,
       1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008,
       2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
       2020, 2021, 2022, 2023, 2024, 2025])

In [8]:
Nt=np.size(yrs)*12
print(Nt)

732


In [9]:
Ny=180
Nx=360
for alg in [2]:
    for argo in [1]:
        for vin in np.arange(2,3,1):
            version,gver = set_version(alg,argo,vin,TSdata)
            print(gver)
            tmp0 = np.zeros((Nt,Nz1,Ny,Nx))
            for n,ver in enumerate(version):
                print(ver)
                fn=f'{diro2}PO4map_v{ver}.nc'
                ds=xr.open_dataset(fn)
                tmp = ds.po4est.to_numpy()
                tmp0 = np.where(np.isnan(tmp),tmp0,tmp)
                #---
            print('cleaning up the data')
            o2=np.where(tmp0==0,np.nan,tmp0)
            print('xarray dataarray')
            da=xr.DataArray(data=o2,name='po4',dims=['time','depth','lat','lon'],coords={'time':ds.time,'depth':z1,'lat':ds.lat,'lon':ds.lon})
            ds=da.to_dataset()
            print('saving in netCDF format, '+diro2+f'PO4map_v{gver}.nc')
            ds.to_netcdf(diro2+f'PO4map_v{gver}.nc')

1.3.G.4.6.4
1.3.1.4.6.4
1.3.2.4.6.4
1.3.3.4.6.4
1.3.4.4.6.4
1.3.5.4.6.4
cleaning up the data
xarray dataarray
saving in netCDF format, /glade/derecho/scratch/ito/ML4O2_results/O2map_v1.3.G.4.6.4.nc
